In [1]:
import numpy as np
import cv2
import keras
from PIL import Image
import os
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path.cwd() / 'src'))

print("Libraries imported successfully!")


Libraries imported successfully!


In [2]:
# Import the custom model class
from training.model import DigitClassifier

# Create a new model instance and load weights
print("Creating new model instance...")
model = DigitClassifier(input_shape=(28, 28, 1), num_classes=10)

# Build the model
sample_input = keras.Input(shape=(28, 28, 1))
_ = model(sample_input)

# Load the weights
model_path = 'models/digit_classifier.h5'
print(f"Loading weights from {model_path}...")

# Try to load just the weights
try:
    model.load_weights(model_path)
    print("✅ Weights loaded successfully!")
except Exception as e:
    print(f"❌ Error loading weights: {e}")
    print("Trying alternative approach...")
    
    # Alternative: Load the full model with compile=False
    try:
        model = keras.models.load_model(model_path, compile=False)
        print("✅ Model loaded successfully (without compilation)!")
    except Exception as e2:
        print(f"❌ Alternative approach failed: {e2}")
        print("Please retrain the model with the updated DigitClassifier class.")

# Print model information
print(f"Model input shape: (28, 28, 1)")
print(f"Model output shape: (10,) - 10 classes (0-9)")
print(f"Model ready for predictions!")


Creating new model instance...
Loading weights from models/digit_classifier.h5...
✅ Weights loaded successfully!
Model input shape: (28, 28, 1)
Model output shape: (10,) - 10 classes (0-9)
Model ready for predictions!


In [3]:
def preprocess_cell_image(image_path, target_size=(28, 28)):
    """
    Preprocess a cell image for prediction.
    """
    # Load image
    img = Image.open(image_path)
    
    # Convert to grayscale if needed
    if img.mode != 'L':
        img = img.convert('L')
    
    # Resize to target size
    img = img.resize(target_size, Image.Resampling.LANCZOS)
    
    # Convert to numpy array
    img_array = np.array(img, dtype=np.float32)
    
    # Normalize to [0, 1]
    img_array = img_array / 255.0
    
    # Add channel dimension
    img_array = np.expand_dims(img_array, axis=-1)
    
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def predict_digit(model, image_path):
    """
    Predict digit from cell image.
    Returns: digit (0-9, where 0 means empty/blank)
    """
    # Preprocess image
    processed_img = preprocess_cell_image(image_path)
    
    # Make prediction
    prediction = model.predict(processed_img, verbose=0)
    
    # Get the predicted class
    predicted_class = np.argmax(prediction[0])
    
    # Get confidence
    confidence = np.max(prediction[0])
    
    return predicted_class, confidence

print("Helper functions defined!")


Helper functions defined!


In [4]:
def populate_sudoku_grid(cells_dir='cells', model=None):
    """
    Populate a 9x9 Sudoku grid by predicting digits from cell images.
    """
    # Initialize 9x9 grid with zeros (empty cells)
    sudoku_grid = np.zeros((9, 9), dtype=int)
    confidence_grid = np.zeros((9, 9), dtype=float)
    
    # Process each cell image
    for i in range(81):  # 9x9 = 81 cells
        # Calculate row and column from cell index
        row = i // 9
        col = i % 9
        
        # Construct filename
        filename = f"cell_{i:02d}_row{row}_col{col}.jpg"
        image_path = os.path.join(cells_dir, filename)
        
        if os.path.exists(image_path):
            # Predict digit
            digit, confidence = predict_digit(model, image_path)
            
            # Store in grid (0 means empty, 1-9 are actual digits)
            sudoku_grid[row, col] = digit
            confidence_grid[row, col] = confidence
            
            print(f"Cell {i:2d} (row {row}, col {col}): {filename} -> Digit: {digit}, Confidence: {confidence:.3f}")
        else:
            print(f"Cell {i:2d} (row {row}, col {col}): {filename} -> NOT FOUND")
    
    return sudoku_grid, confidence_grid

print("Grid population function defined!")


Grid population function defined!


In [5]:
def display_sudoku_grid(grid, title="Sudoku Grid"):
    """
    Display the Sudoku grid in a nice format.
    """
    print(f"\n{title}")
    print("=" * 25)
    
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 25)
        
        row_str = ""
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row_str += "| "
            
            if grid[i, j] == 0:
                row_str += ". "
            else:
                row_str += f"{grid[i, j]} "
        
        print(row_str)
    
    print("=" * 25)

def display_confidence_grid(confidence_grid, title="Confidence Grid"):
    """
    Display the confidence grid.
    """
    print(f"\n{title}")
    print("=" * 25)
    
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 25)
        
        row_str = ""
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row_str += "| "
            
            row_str += f"{confidence_grid[i, j]:.2f} "
        
        print(row_str)
    
    print("=" * 25)

print("Display functions defined!")


Display functions defined!


In [6]:
# Populate the Sudoku grid using the trained model
print("Starting Sudoku grid population...")
print("Processing 81 cell images...")

sudoku_grid, confidence_grid = populate_sudoku_grid('cells', model)

print(f"\nCompleted! Processed {np.sum(sudoku_grid > 0)} non-empty cells out of 81 total cells.")


Starting Sudoku grid population...
Processing 81 cell images...
Cell  0 (row 0, col 0): cell_00_row0_col0.jpg -> Digit: 8, Confidence: 0.314
Cell  1 (row 0, col 1): cell_01_row0_col1.jpg -> Digit: 8, Confidence: 0.378
Cell  2 (row 0, col 2): cell_02_row0_col2.jpg -> Digit: 1, Confidence: 0.306
Cell  3 (row 0, col 3): cell_03_row0_col3.jpg -> Digit: 8, Confidence: 0.340
Cell  4 (row 0, col 4): cell_04_row0_col4.jpg -> Digit: 8, Confidence: 0.380
Cell  5 (row 0, col 5): cell_05_row0_col5.jpg -> Digit: 8, Confidence: 0.381
Cell  6 (row 0, col 6): cell_06_row0_col6.jpg -> Digit: 8, Confidence: 0.349
Cell  7 (row 0, col 7): cell_07_row0_col7.jpg -> Digit: 1, Confidence: 0.371
Cell  8 (row 0, col 8): cell_08_row0_col8.jpg -> Digit: 1, Confidence: 0.388
Cell  9 (row 1, col 0): cell_09_row1_col0.jpg -> Digit: 8, Confidence: 0.328
Cell 10 (row 1, col 1): cell_10_row1_col1.jpg -> Digit: 8, Confidence: 0.369
Cell 11 (row 1, col 2): cell_11_row1_col2.jpg -> Digit: 1, Confidence: 0.375
Cell 12 (row

In [7]:
# Display the results
display_sudoku_grid(sudoku_grid, "Predicted Sudoku Grid")
display_confidence_grid(confidence_grid, "Prediction Confidence")

# Summary statistics
print(f"\nSummary:")
print(f"Total cells: 81")
print(f"Empty cells (0): {np.sum(sudoku_grid == 0)}")
print(f"Filled cells (1-9): {np.sum(sudoku_grid > 0)}")
print(f"Average confidence: {np.mean(confidence_grid[sudoku_grid > 0]):.3f}")
print(f"Min confidence: {np.min(confidence_grid[sudoku_grid > 0]):.3f}")
print(f"Max confidence: {np.max(confidence_grid[sudoku_grid > 0]):.3f}")



Predicted Sudoku Grid
8 8 1 | 8 8 8 | 8 1 1 
8 8 1 | 1 1 1 | 8 1 8 
8 1 1 | 8 8 1 | 1 1 8 
-------------------------
8 8 8 | 8 1 8 | 8 8 8 
8 8 8 | 8 8 1 | 1 8 1 
8 1 8 | 8 8 8 | 1 1 1 
-------------------------
8 8 1 | 8 8 8 | 8 8 8 
1 8 8 | 8 8 8 | 8 8 8 
1 1 8 | 8 8 8 | 8 8 8 

Prediction Confidence
0.31 0.38 0.31 | 0.34 0.38 0.38 | 0.35 0.37 0.39 
0.33 0.37 0.37 | 0.33 0.32 0.37 | 0.38 0.35 0.31 
0.37 0.38 0.37 | 0.38 0.38 0.37 | 0.34 0.33 0.40 
-------------------------
0.42 0.38 0.42 | 0.32 0.37 0.32 | 0.41 0.38 0.38 
0.40 0.40 0.34 | 0.38 0.33 0.39 | 0.38 0.34 0.35 
0.39 0.32 0.34 | 0.39 0.36 0.31 | 0.32 0.36 0.36 
-------------------------
0.36 0.40 0.38 | 0.37 0.39 0.31 | 0.38 0.41 0.39 
0.31 0.41 0.39 | 0.41 0.37 0.43 | 0.44 0.41 0.36 
0.39 0.39 0.37 | 0.37 0.34 0.36 | 0.40 0.38 0.41 

Summary:
Total cells: 81
Empty cells (0): 0
Filled cells (1-9): 81
Average confidence: 0.368
Min confidence: 0.306
Max confidence: 0.439


In [8]:
# Save the results
np.save('predicted_sudoku_grid.npy', sudoku_grid)
np.save('prediction_confidence.npy', confidence_grid)

print("Results saved:")
print("- predicted_sudoku_grid.npy")
print("- prediction_confidence.npy")

# Show the raw numpy array for verification
print(f"\nRaw Sudoku grid (numpy array):")
print(sudoku_grid)


Results saved:
- predicted_sudoku_grid.npy
- prediction_confidence.npy

Raw Sudoku grid (numpy array):
[[8 8 1 8 8 8 8 1 1]
 [8 8 1 1 1 1 8 1 8]
 [8 1 1 8 8 1 1 1 8]
 [8 8 8 8 1 8 8 8 8]
 [8 8 8 8 8 1 1 8 1]
 [8 1 8 8 8 8 1 1 1]
 [8 8 1 8 8 8 8 8 8]
 [1 8 8 8 8 8 8 8 8]
 [1 1 8 8 8 8 8 8 8]]
